In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_csv("raw_ksi_data.csv")

In [3]:
df.shape

(18957, 54)

In [4]:
df.sample(2)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
6050,6051,6160318,1176502.0,7/8/2010 8:00:00 AM,1725,EGLINTON AVE E,BIMBROK RD,NaN,Major Arterial,Scarborough,...,NaN,NaN,NaN,124,Kennedy Park,124,Kennedy Park (124),D41,640814.6313,4.844030e+06
1662,1663,4140864,956470.0,2/16/2007 10:00:00 AM,2305,4400 DUFFERIN Stre,NaN,NaN,Major Arterial,North York,...,Yes,NaN,NaN,27,York University Heights,27,York University Heights (27),D32,623456.2106,4.846964e+06


In [5]:
df.drop(['ACCNUM','OBJECTID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'INVAGE', 'INITDIR'], axis=1, inplace=True)

In [6]:
df.drop(['VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', "TRSN_CITY_VEH"], axis=1, inplace=True)

In [7]:
df.drop(['EMERG_VEH', 'PASSENGER', 'REDLIGHT', 'DISABILITY', 'DIVISION', 'SPEEDING', 'PEDCOND', 'AG_DRIV','x', 'y' ], axis=1, inplace=True)

In [8]:
df.drop(['CYCLIST', 'IMPACTYPE', 'INVTYPE', 'INDEX', 'TRAFFCTL', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'ALCOHOL'], axis=1, inplace=True)

In [9]:
df.shape

(18957, 15)

In [10]:
df.sample(2)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158
5715,4/3/2010 8:00:00 AM,159,ALBION Rd,HIGHWAY 27 N,Etobicoke York,43.744245,-79.607490,Clear,Dark,Dry,Non-Fatal Injury,Minor,NaN,1,West Humber-Clairville
13216,12/21/2016 10:00:00 AM,1340,F G GARDINER XY W,JAMESON AVE,Toronto and East York,43.633063,-79.434955,Clear,Daylight,Dry,Non-Fatal Injury,NaN,NaN,85,South Parkdale


In [11]:
df['TIME'] = df['TIME'].astype(str).str.zfill(4)
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract DOW and HOUR and YEAR
df['DOW'] = df['DATE'].dt.day_name() 
df['HOUR'] = df['TIME'].str[:2].astype(int)
df['YEAR'] = df['DATE'].dt.year

In [12]:
df.sample(3)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR
17771,2022-06-27 08:00:00,0238,QUEEN ST W,BELLWOODS AVE,Toronto and East York,43.645774,-79.410747,Clear,"Dark, artificial",Dry,Non-Fatal Injury,NaN,NaN,162,West Queen West,Monday,2,2022
6047,2010-07-08 08:00:00,1429,1048 BLOOR ST W,NaN,Toronto and East York,43.660545,-79.432290,Clear,Daylight,Dry,Non-Fatal Injury,NaN,NaN,172,Dovercourt Village,Thursday,14,2010
12144,2015-11-27 10:00:00,0532,EGLINTON AVE W,BATHURST ST,North York,43.700984,-79.425548,Clear,Dark,Dry,Fatal,NaN,NaN,101,Forest Hill South,Friday,5,2015


In [13]:
df["INJURY"].unique()

array(['Major', 'Minor', nan, 'Fatal', 'Minimal'], dtype=object)

In [14]:
df["INJURY"].value_counts()

INJURY
Major      6445
Minor      1479
Minimal    1160
Fatal       976
Name: count, dtype: int64

In [15]:
df['INJURY'] = df['INJURY'].fillna('Minimal')

In [16]:
df["ACCLASS"].unique()

array(['Non-Fatal Injury', 'Fatal', 'Property Damage O', nan],
      dtype=object)

In [17]:
df["ACCLASS"].value_counts()

ACCLASS
Non-Fatal Injury     16268
Fatal                 2670
Property Damage O       18
Name: count, dtype: int64

In [18]:
df['ACCLASS'] = df['ACCLASS'].replace('Property Damage O', 'Non-Fatal Injury')
df['ACCLASS'] = df['ACCLASS'].fillna('Non-Fatal Injury')

In [19]:
df['SEVERE_COLLISION'] = (
    (df['INJURY'].isin(['Major', 'Fatal'])) | 
    (df['ACCLASS'] == 'Fatal')
).astype(int)

In [20]:
df['FATAL_NO'] = df['FATAL_NO'].fillna(0)

In [21]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear                     16373
Rain                       1976
Snow                        356
Other                        98
Fog, Mist, Smoke, Dust       52
Freezing Rain                47
Drifting Snow                21
Strong wind                  10
Name: count, dtype: int64

In [22]:
df['VISIBILITY'] = df['VISIBILITY'].replace({
    'Other': 'Clear',
    'Freezing Rain': 'Rain',
    'Strong wind': 'Rain',
    'Fog, Mist, Smoke, Dust': 'Rain',
    'Drifting Snow': 'Snow'
})

In [23]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear    16471
Rain      2085
Snow       377
Name: count, dtype: int64

In [24]:
df["LIGHT"].value_counts()

LIGHT
Daylight                10779
Dark                     3746
Dark, artificial         3552
Dusk                      253
Dusk, artificial          253
Daylight, artificial      151
Dawn                      112
Dawn, artificial          101
Other                       6
Name: count, dtype: int64

In [25]:
df['LIGHT'] = df['LIGHT'].replace({
    'Daylight, artificial': 'Daylight',
    'Dark, artificial': 'Dark',
    'Dusk, artificial': 'Dusk',
    'Dawn, artificial': 'Dawn',
    'Other': 'Daylight'
})

In [26]:
df["LIGHT"].value_counts()

LIGHT
Daylight    10936
Dark         7298
Dusk          506
Dawn          213
Name: count, dtype: int64

In [27]:
df.rename(columns={'RDSFCOND': 'ROAD_CONDITION'}, inplace=True)

In [28]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry                     15231
Wet                      3140
Loose Snow                174
Other                     147
Slush                     102
Ice                        78
Packed Snow                44
Loose Sand or Gravel       11
Spilled liquid              1
Name: count, dtype: int64

In [29]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace({
    'Spilled liquid': 'Wet',
    'Loose Snow': 'Snow_Ice',
    'Packed Snow': 'Snow_Ice',
    'Slush': 'Snow_Ice',
    'Ice': 'Snow_Ice',
    'Loose Sand or Gravel': 'Dry',
    'Other': 'Dry'
})

In [30]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry         15389
Wet          3141
Snow_Ice      398
Name: count, dtype: int64

In [31]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

In [32]:
df.drop(['TIME'], axis=1, inplace=True)#since hour col, exact time not needed

In [33]:
df.sample(5)

,DATE,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,ROAD_CONDITION,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,SEVERE_COLLISION
10437,2014-01-30,HIGHWAY 27 S,CARRIER DR,Etobicoke York,43.739615,-79.605655,Clear,Daylight,Dry,Non-Fatal Injury,Major,0.0,1,West Humber-Clairville,Thursday,10,2014,1
3317,2008-05-04,DUFFERIN Stre,LAWRENCE Aven W,North York,43.714545,-79.454790,Clear,Dark,Dry,Non-Fatal Injury,Major,0.0,31,Yorkdale-Glen Park,Sunday,23,2008,1
418,2006-05-11,MEADOWVALE RD,PLUG HAT RD,Scarborough,43.835745,-79.182990,Rain,Daylight,Wet,Non-Fatal Injury,Minimal,0.0,144,Morningside Heights,Thursday,6,2006,0
10726,2014-05-31,F G GARDINER XY W,SPADINA AVE,Toronto and East York,43.638892,-79.390979,Clear,Dark,Dry,Non-Fatal Injury,Minimal,0.0,165,Harbourfront-CityPlace,Saturday,22,2014,0
9453,2013-04-24,MORNINGSIDE Ave,BEATH St,Scarborough,43.774745,-79.188690,Rain,Daylight,Wet,Non-Fatal Injury,Minor,0.0,136,West Hill,Wednesday,16,2013,0


In [34]:
df.shape

(18957, 18)

In [35]:
df.to_csv('cleaned_ksi_collisions.csv', index=False)